# Gabor's Neighborhoods in Toronto project  


## Part 1: Wikipedia data pull and manipulation

In [18]:
import pandas as pd 
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
print('Import done.')

Import done.


In [19]:
# read in the html from website
table = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
d = table[0]

In [20]:
# create a dataframe
data = d[['Postal code','Borough','Neighborhood']]
data.head(3)

,Postal code,Borough,Neighborhood
0,M1A,Not assigned,NaN
1,M2A,Not assigned,NaN
2,M3A,North York,Parkwoods


In [21]:
# Let's see how many lines are there in total
data.shape

(180, 3)

In [22]:
# Let's see how many "Not assigned" has to be dropped
data['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
York                 5
East Toronto         5
East York            5
Mississauga          1
Name: Borough, dtype: int64

In [23]:
# Filter out the 77 lines with "Not assigned"
condition = data['Borough'] != 'Not assigned'
data2 = data[condition]
data2.head(3)

,Postal code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Regent Park / Harbourfront


**COMMENT:**  
These below actions I could not perform as M5A was not listed twice and after the Not assigned Borough removal there were no blank Neighborhoods at all. 

1. More than one neighborhood can exist in one postal code area. For example, in the table on the Wikipedia page, you will notice that M5A is listed twice and has two neighborhoods: Harbourfront and Regent Park. These two rows will be combined into one row with the neighborhoods separated with a comma as shown in row 11 in the above table.

2. If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.

In [24]:
# data2['Neighborhood'].value_counts()                 un-hashtag to see there is no Not assigned neighborhood

In [25]:
# Double-check to see if 180-77= 103 lines remained
data2.shape

(103, 3)

## Part 2: Adding geo data

In [27]:
# read in the geo data from website
geos = pd.read_csv('http://cocl.us/Geospatial_data')
geos.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [29]:
left = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],'A': ['A0', 'A1', 'A2', 'A3'],'B': ['B0', 'B1', 'B2', 'B3']})
right = pd.DataFrame({'key': ['K0', 'K1', 'K2', 'K3'],'C': ['C0', 'C1', 'C2', 'C3'],'D': ['D0', 'D1', 'D2', 'D3']})
result = pd.merge(left, right, on='key')
result

,key,A,B,C,D
0,K0,A0,B0,C0,D0
1,K1,A1,B1,C1,D1
2,K2,A2,B2,C2,D2
3,K3,A3,B3,C3,D3


In [39]:
# merging the tables based on the postal code
key = data2['Postal code']
merged = pd.merge(data2,geos, on=key)

#drop the unnecessary key column
merged.drop("key_0", axis = 1, inplace=True)

merged.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
0,M3A,North York,Parkwoods,M1B,43.806686,-79.194353
1,M4A,North York,Victoria Village,M1C,43.784535,-79.160497
2,M5A,Downtown Toronto,Regent Park / Harbourfront,M1E,43.763573,-79.188711
3,M6A,North York,Lawrence Manor / Lawrence Heights,M1G,43.770992,-79.216917
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M1H,43.773136,-79.239476


*Explore and cluster the neighborhoods in Toronto. **You can decide to work with only boroughs that contain the word Toronto** and then replicate the same analysis we did to the New York City data.* 

In [44]:
# Filter out the Boroughs with "Toronto"
condition = merged['Borough'].str.contains('Toronto')    
merged2 = merged[condition]
merged2.head()

,Postal code,Borough,Neighborhood,Postal Code,Latitude,Longitude
2,M5A,Downtown Toronto,Regent Park / Harbourfront,M1E,43.763573,-79.188711
4,M7A,Downtown Toronto,Queen's Park / Ontario Provincial Government,M1H,43.773136,-79.239476
9,M5B,Downtown Toronto,"Garden District, Ryerson",M1N,43.692657,-79.264848
15,M5C,Downtown Toronto,St. James Town,M1W,43.799525,-79.318389
19,M4E,East Toronto,The Beaches,M2K,43.786947,-79.385975


In [50]:
# See the size of the df
merged2.shape

(39, 6)

In [ ]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')